In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate();

using Flux, Flux.Data.MNIST, Statistics
using Flux: throttle, params
using Juno: @progress

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


Extend distributions slightly to have a numerically stable logpdf for `p` close to 1 or 0.

In [2]:
using Distributions
import Distributions: logpdf
logpdf(b::Bernoulli, y::Bool) = y * log(b.p + eps()) + (1 - y) * log(1 - b.p + eps())

┌ Warning: Module StatsBase with build ID 1459188128752384 is missing from the cache.
│ This may mean StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Warning: Module SpecialFunctions with build ID 1450108083806226 is missing from the cache.
│ This may mean SpecialFunctions [276daf66-3868-5448-9aa4-cd146d93841b] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Warning: Module DataStructures with build ID 1459191013855216 is missing from the cache.
│ This may mean DataStructures [864edb3b-99cc-5e75-8d2d-829cb0a9cfe8] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947


logpdf (generic function with 62 methods)

Load data, binarise it, and partition into mini-batches of M.

In [3]:
X = float.(hcat(vec.(MNIST.images())...)) .> 0.5
N, M = size(X, 2), 100
data = [X[:,i] for i in Iterators.partition(1:N,M)]


################################# Define Model #################################

600-element Array{BitArray{2},1}:
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false

Latent dimensionality, # hidden units.

In [4]:
Dz, Dh = 5, 500

(5, 500)

Components of recognition model / "encoder" MLP.

In [5]:
A, μ, logσ = Dense(28^2, Dh, tanh), Dense(Dh, Dz), Dense(Dh, Dz)
g(X) = (h = A(X); (μ(h), logσ(h)))
z(μ, logσ) = μ + exp(logσ) * randn()

z (generic function with 1 method)

Generative model / "decoder" MLP.

In [6]:
f = Chain(Dense(Dz, Dh, tanh), Dense(Dh, 28^2, σ))


####################### Define ways of doing things with the model. #######################

Chain(Dense(5, 500, tanh), Dense(500, 784, NNlib.σ))

KL-divergence between approximation posterior and N(0, 1) prior.

In [7]:
kl_q_p(μ, logσ) = 0.5 * sum(exp.(2 .* logσ) + μ.^2 .- 1 .+ logσ.^2)

kl_q_p (generic function with 1 method)

logp(x|z) - conditional probability of data given latents.

In [8]:
logp_x_z(x, z) = sum(logpdf.(Bernoulli.(f(z)), x))

logp_x_z (generic function with 1 method)

Monte Carlo estimator of mean ELBO using M samples.

In [9]:
L̄(X) = ((μ̂, logσ̂) = g(X); (logp_x_z(X, z.(μ̂, logσ̂)) - kl_q_p(μ̂, logσ̂)) / M)

loss(X) = -L̄(X) + 0.01 * sum(x->sum(x.^2), params(f))

loss (generic function with 1 method)

Sample from the learned model.

In [10]:
modelsample() = rand.(Bernoulli.(f(z.(zeros(Dz), zeros(Dz)))))


################################# Learn Parameters ##############################

evalcb = throttle(() -> @show(-L̄(X[:, rand(1:N, M)])), 30)
opt = ADAM(params(A, μ, logσ, f))
@progress for i = 1:20
  @info "Epoch $i"
  Flux.train!(loss, zip(data), opt, cb=evalcb)
end


################################# Sample Output ##############################

using Images

img(x) = Gray.(reshape(x, 28, 28))

cd(@__DIR__)
sample = hcat(img.([modelsample() for i = 1:10])...)
save("sample.png", sample)

┌ Info: Epoch 1
└ @ Main.##738 string:9
-(L̄(X[:, rand(1:N, M)])) = 551.0339316170952 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 197.36916138186595 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 175.6337985107018 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 167.03643765859331 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 171.57124888549475 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 180.4850619905305 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 170.97635744531388 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 168.35696497746406 (tracked)
┌ Info: Epoch 2
└ @ Main.##738 string:9
-(L̄(X[:, rand(1:N, M)])) = 172.2982575002878 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 170.99741622490492 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 166.96967988360845 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 161.38892883793972 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 159.28501341334854 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 162.0019334718976 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 154.2080511727885 (tracked)
┌ Info: Epoch 3
└ @ Main.##738 string:9
-(L̄(X[:, rand(1:N, M)])) = 158